In [126]:
import csv
import numpy as np
import nltk

nltk.download('stopwords')

seed =  42
MAX_SENTENCES = 20000

[nltk_data] Downloading package stopwords to /home/maciek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
dataset = []
dataset_x = []
dataset_y = []

labels = {0:0, 4:1}

with open('training.1600000.processed.noemoticon.csv', 'r', encoding='ISO-8859-1') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        dataset.append((labels[int(line[0])], line[5]))
    rng = np.random.RandomState(seed)
    ids = rng.choice(len(dataset), MAX_SENTENCES)
    smaller_dataset = [dataset[i] for i in ids]
    dataset_x = [d[1] for d in smaller_dataset]
    dataset_y = [d[0] for d in smaller_dataset]

print(len(dataset_x))
        
def print_n(data_x, data_y, n=10):
    for x,y in zip(data_x[:n], data_y[:n]):
        print(y,x)
        
print_n(dataset_x, dataset_y)

20000
0 @ home studying for maths wooot ! im so going to fail this shit 
0 Pickin up @misstinayao waitin on @sadittysash 2 hurry up...I odeeee missed dem  Table talk 2nite...LOL bout to be fat...
0 @ProudGamerTweet I rather average 32370 
1 @officialnjonas Good luck with that 
0 this song's middle change just doesn't want to be born..... arghhhh!! 
0 im starting my many hours of work now 
0 Thunderstorms yesterday, more on the way. Looks like I won't be online much again today.  HAPPY FATHER'S DAY
1 @cloecouturier Yes, I do have a few ..   4 Girls ...   You son is 23 ... all grown up now .. happens fast!!
0 Last free travel at AP-1  http://yfrog.com/0uvu5j
1 TF2 has updated but its way to late to play.  looks like I'll be spying and sniping my heart out tomorrow 


In [128]:
import re

def preprocess_regex(sentence):
    sentence = re.sub('[^a-z\',.?! ]', ' ', sentence.lower())
    sentence = re.sub('\s+', ' ', sentence)
    return sentence.strip()
    
dataset_x_clear = [preprocess_regex(x) for x in dataset_x]

print_n(dataset_x_clear, dataset_y)

0 home studying for maths wooot ! im so going to fail this shit
0 pickin up misstinayao waitin on sadittysash hurry up...i odeeee missed dem table talk nite...lol bout to be fat...
0 proudgamertweet i rather average
1 officialnjonas good luck with that
0 this song's middle change just doesn't want to be born..... arghhhh!!
0 im starting my many hours of work now
0 thunderstorms yesterday, more on the way. looks like i won't be online much again today. happy father's day
1 cloecouturier yes, i do have a few .. girls ... you son is ... all grown up now .. happens fast!!
0 last free travel at ap http yfrog.com uvu j
1 tf has updated but its way to late to play. looks like i'll be spying and sniping my heart out tomorrow


In [129]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

tokenizer = TweetTokenizer().tokenize
stemmer = PorterStemmer().stem

def tokenize_sentence(sentence):
    sentence = tokenizer(sentence)
    sentence = [stemmer(word) for word in sentence if len(word) > 2]
    return sentence

dataset_x_tokenized = [tokenize_sentence(x) for x in dataset_x_clear]

print_n(dataset_x_tokenized, dataset_y)

0 ['home', 'studi', 'for', 'math', 'wooot', 'go', 'fail', 'thi', 'shit']
0 ['pickin', 'misstinayao', 'waitin', 'sadittysash', 'hurri', '...', 'odeee', 'miss', 'dem', 'tabl', 'talk', 'nite', '...', 'lol', 'bout', 'fat', '...']
0 ['proudgamertweet', 'rather', 'averag']
1 ['officialnjona', 'good', 'luck', 'with', 'that']
0 ['thi', "song'", 'middl', 'chang', 'just', "doesn't", 'want', 'born', '...', 'arghhhh']
0 ['start', 'mani', 'hour', 'work', 'now']
0 ['thunderstorm', 'yesterday', 'more', 'the', 'way', 'look', 'like', "won't", 'onlin', 'much', 'again', 'today', 'happi', "father'", 'day']
1 ['cloecouturi', 'ye', 'have', 'few', 'girl', '...', 'you', 'son', '...', 'all', 'grown', 'now', 'happen', 'fast']
0 ['last', 'free', 'travel', 'http', 'yfrog.com', 'uvu']
1 ['ha', 'updat', 'but', 'it', 'way', 'late', 'play', 'look', 'like', "i'll", 'spi', 'and', 'snipe', 'heart', 'out', 'tomorrow']


In [130]:
from nltk.corpus import stopwords

stops = stopwords.words('english')

def remove_stopwords(sentence):
    sentence = [word for word in sentence if word not in stops]
    return ' '.join(sentence)

dataset_x_no_stopwords = [remove_stopwords(x) for x in dataset_x_tokenized]

print_n(dataset_x_no_stopwords, dataset_y)

0 home studi math wooot go fail thi shit
0 pickin misstinayao waitin sadittysash hurri ... odeee miss dem tabl talk nite ... lol bout fat ...
0 proudgamertweet rather averag
1 officialnjona good luck
0 thi song' middl chang want born ... arghhhh
0 start mani hour work
0 thunderstorm yesterday way look like onlin much today happi father' day
1 cloecouturi ye girl ... son ... grown happen fast
0 last free travel http yfrog.com uvu
1 ha updat way late play look like i'll spi snipe heart tomorrow


In [131]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,3), max_features=5000)
vectorizer.fit(dataset_x_no_stopwords)

dataset_x_bow = vectorizer.transform(dataset_x_no_stopwords)

print(dataset_x_bow.shape)

(20000, 5000)


In [132]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(dataset_x_bow)
dataset_x_tfidf = tf_transformer.transform(dataset_x_bow)

print(dataset_x_tfidf.shape)

(20000, 5000)


In [133]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=seed,solver='lbfgs',
                          multi_class='multinomial').fit(dataset_x_tfidf, dataset_y)

In [134]:
test_x = []
test_y = []

with open('testdata.manual.2009.06.14.csv', 'r', encoding='ISO-8859-1') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if int(line[0]) != 2:
            test_y.append(labels[int(line[0])])
            test_x.append(line[5])
        
def process_sentences(sentences):
    sentences = [preprocess_regex(s) for s in sentences]
    sentences = [tokenize_sentence(s) for s in sentences]
    sentences = [remove_stopwords(s) for s in sentences]
    sentences = vectorizer.transform(sentences)
    sentences = tf_transformer.transform(sentences)
    return sentences

test_x_processed = process_sentences(test_x)

print(test_x_processed.shape)
        

(359, 5000)


In [136]:
score = clf.score(test_x_processed, test_y)
print("Accuracy:", score)

Accuracy: 0.7771587743732591


In [143]:
sentences = [
    'Volvo IT days is an awesome conference!',
    'I spoke at Volvo IT days',
    "It's a shame that my speach took only 30 mins!"
]

def get_label(proba, treshold=0.1):
    if abs(proba[0]-proba[1]) < treshold:
        return 'not sure'
    else:
        return 'positive' if proba[0] < proba[1] else 'negative'

sentences_processed = process_sentences(sentences)

probas = clf.predict_proba(sentences_processed)

for sent, proba in zip(sentences, probas):
    print(get_label(proba), ',', proba, ',', sent)

positive , [0.24013723 0.75986277] , Volvo IT days is an awesome conference!
not sure , [0.46271552 0.53728448] , I spoke at Volvo IT days
negative , [0.70568241 0.29431759] , It's a shame that my speach took only 30 mins!
